In [2]:
from utils import *
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time

In [11]:
# ! rm -rf data/negative/.ipynb_checkpoints

In [3]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8',)
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [4]:
ONEHOT = np.zeros((len(trainset.data),len(trainset.target_names)))
ONEHOT[np.arange(len(trainset.data)),trainset.target] = 1.0
train_X, test_X, train_Y, test_Y, train_onehot, test_onehot = train_test_split(trainset.data, 
                                                                               trainset.target, 
                                                                               ONEHOT, test_size = 0.2)

In [5]:
train_X[:2]

['4 friends 2 couples 2000 miles pabst blue ribbon beer drink ultimate redneck roadtrip',
 'bit unwieldy mess']

In [6]:
train_onehot[:2]

array([[0., 1.],
       [1., 0.]])

In [7]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))

data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)

print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [1122, 719, 109, 2417, 2723, 4373, 5, 73, 40, 7006] ['awkward', 'sincere', 'ultimately', 'wins', 'undoubtedly', 'scariest', 'movie', 'ever', 'made', 'tattoos']


In [8]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [9]:
class Model:
    def __init__(self, size_layer, num_layers, embedded_size,
                 dict_size, dimension_output, learning_rate):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.BasicRNNCell(size_layer,reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.float32, [None, dimension_output])
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
        self.outputs, _ = tf.nn.dynamic_rnn(rnn_cells, encoder_embedded, dtype = tf.float32)
        W = tf.get_variable('w',shape=(size_layer, dimension_output),initializer=tf.orthogonal_initializer())
        b = tf.get_variable('b',shape=(dimension_output),initializer=tf.zeros_initializer())
        
        #对于一个三维矩阵a,a[:,-1]相当于a[:,-1,:]
        #该模型分类方法是，序列的最后一个词向量特征作为句子向量的特征表示，使用全连接层进行分类
        self.logits = tf.matmul(self.outputs[:, -1], W) + b
        
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [10]:
size_layer = 128
num_layers = 2
embedded_size = 128
dimension_output = len(trainset.target_names)
learning_rate = 1e-3
maxlen = 50
batch_size = 128

In [11]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer,num_layers,embedded_size,vocabulary_size+4,dimension_output,learning_rate)
sess.run(tf.global_variables_initializer())

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [12]:
# 练习
a = np.arange(24).reshape(2,3,4)
print(a)
print(a[:,0])

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
[[ 0  1  2  3]
 [12 13 14 15]]


In [15]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(train_X[i:i+batch_size],dictionary,maxlen)
        
        feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]}

        
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(test_X[i:i+batch_size],dictionary,maxlen)
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : batch_x, model.Y : test_onehot[i:i+batch_size]})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.543457
time taken: 2.196401357650757
epoch: 0, training loss: 0.707161, training acc: 0.524384, valid loss: 0.691960, valid acc: 0.543457

epoch: 1, pass acc: 0.543457, current acc: 0.577637
time taken: 1.9032156467437744
epoch: 1, training loss: 0.627698, training acc: 0.648674, valid loss: 0.700298, valid acc: 0.577637

epoch: 2, pass acc: 0.577637, current acc: 0.620117
time taken: 1.8803789615631104
epoch: 2, training loss: 0.511525, training acc: 0.747988, valid loss: 0.734392, valid acc: 0.620117

epoch: 3, pass acc: 0.620117, current acc: 0.638184
time taken: 1.916853427886963
epoch: 3, training loss: 0.367111, training acc: 0.835701, valid loss: 0.860722, valid acc: 0.638184

epoch: 4, pass acc: 0.638184, current acc: 0.645508
time taken: 1.9410135746002197
epoch: 4, training loss: 0.238095, training acc: 0.904948, valid loss: 1.163005, valid acc: 0.645508

epoch: 5, pass acc: 0.645508, current acc: 0.663574
time taken: 1.97587895393

In [18]:
logits = sess.run(model.logits, feed_dict={model.X:str_idx(test_X,dictionary,maxlen)})
print(metrics.classification_report(test_Y, np.argmax(logits,1), target_names = trainset.target_names))

              precision    recall  f1-score   support

    negative       0.62      0.67      0.65      1064
    positive       0.65      0.59      0.62      1069

    accuracy                           0.63      2133
   macro avg       0.63      0.63      0.63      2133
weighted avg       0.63      0.63      0.63      2133

